# SPECTRE PROTOTYPE
---

## `producer.py`

`producer.py` is a Python script that performs the following tasks:

1. Import necessary libraries and modules. 
2. Print a welcome message and wait for 1 second. 
3. Define the prod_datapreprocess function, which performs the following tasks: 
   1. Read a CSV file and create a DataFrame.
   2. Clean the dataset by removing NaN, inf, and -inf values.
   3. Perform feature scaling using StandardScaler.
   4. Apply PCA (Principal Component Analysis) to reduce dimensionality.
   5. Combine the principal components with the original labels.
   6. Perform label binarization for the 'label' column.
   7. Return the processed DataFrame with features (X) and labels (y).
4. Call the prod_datapreprocess function on a given dataset.
5. Configure the Kafka producer with the necessary settings.
6. Create a Kafka producer instance.
7. Configure and create a Kafka consumer for handshake purposes.
8. Perform a handshake between the producer and consumer, waiting for a 'READY' message.
9. Send a 'READY' message from the producer to the consumer.
10. Iterate through the processed DataFrame (X) and send each row to the Kafka producer.
11. Flush the producer to ensure all messages are sent.

In [ ]:
# producer.py
# https://www.phind.com/search?cache=cf139efb-38e8-4fb5-9cda-5c67194a11a6

# Import necessary libraries
from confluent_kafka import Producer, Consumer, KafkaError
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.decomposition import PCA
import time

# Print the welcome message
print("==================================")
print("SPECTRE - PRODUCER MODULE")
print("==================================")
time.sleep(1)


# Define a function to preprocess the data
def prod_datapreprocess(csv_file):
    
    # Read a CSV file and create a DataFrame
    df = pd.read_csv(csv_file)
    
    dimensions_num_for_PCA = 7
    
    # Function to clean the dataset by removing NaN, inf, and -inf values
    def clean_dataset(df):
        assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
        df.dropna(inplace=True)
        indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        return df[indices_to_keep]

    # Function to get PCA feature names
    def get_PCA_feature_names(num_of_pca_components):
        feature_names = []
        for i in range(num_of_pca_components):
            feature_names.append(f"Principal component {i+1}")
        return feature_names
    
    # Preprocess the dataset
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df_cleaned = df.copy()
    df_cleaned = clean_dataset(df_cleaned)

    df_cleaned = df_cleaned.reset_index()
    df_cleaned.drop('index', axis=1, inplace=True)

    # Saving the label attribute before dropping it
    df_labels = df_cleaned['label']
    df_cleaned.drop('label', axis=1, inplace=True)
    df_features = df_cleaned.columns.tolist()

    # Perform feature scaling
    df_scaled = StandardScaler().fit_transform(df_cleaned)
    df_scaled = pd.DataFrame(data=df_scaled, columns=df_features)

    # Performing PCA
    pca = PCA(n_components=dimensions_num_for_PCA)
    principal_components = pca.fit_transform(df_scaled)

    # Creating a DataFrame with principal components
    principal_component_headings = get_PCA_feature_names(dimensions_num_for_PCA)
    df_pc = pd.DataFrame(data=principal_components, columns=principal_component_headings)

    # Combine the principal components with the original labels
    df_final = pd.concat([df_pc, df_labels], axis=1)

    # Perform label binarization. Converts "ANOMALY" = 1 and "BENIGN" = 0.
    lb = LabelBinarizer()
    df_final['label'] = lb.fit_transform(df_final['label'])

    # Split the dataset into features (X) and labels (y)
    X = df_final.drop(['label'], axis = 1)
    y = df_final['label']

    # Returns features(X)
    return X


# Read the CSV file and preprocess the data

# DDoS Attack CSV
X = prod_datapreprocess('/home/aryn/spectre-dev/dataset/CICIDS2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

# DDoS Prime CSV
#X = prod_datapreprocess('/home/aryn/spectre-dev/dataset/DDoS_Dataset/ddos_balanced/final_dataset.csv')

# Bening CSV
#X = prod_datapreprocess('/home/aryn/spectre-dev/dataset/CICIDS2017/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv')

# Set the producer configuration
producer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'max.in.flight.requests.per.connection': 1   # Add this line to set the maximum number of in-flight messages to 1
}

# Create a Kafka producer
producer = Producer(producer_conf)

# Set the handshake consumer configuration
handshake_consumer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'producer_handshake_group',
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'
}

# Create a Kafka consumer for the handshake
handshake_consumer = Consumer(handshake_consumer_conf)
handshake_consumer.subscribe(['handshake'])

# Wait for the handshake from the consumer
timeout_counter = 0
timeout_limit = 10

# Perform handshake with the consumer
while True:
    # ... (Handshake waiting code)
    msg = handshake_consumer.poll(1.0)
    if msg is None:
        timeout_counter += 1
        if timeout_counter >= timeout_limit:
            print("==================================")
            print("CONNECTION FAILURE")
            print("==================================")
            exit(1)
        continue
    if msg.error():
        print(f"Handshake consumer error: {msg.error()}")
    else:
        handshake_msg = msg.value().decode('utf-8')
        if handshake_msg == 'READY':
            print("==================================")
            print("CONNECTION ESTABLISHED")
            print("==================================")
            break

# Send a ready message to the consumer
producer.produce('handshake', 'READY')

# Iterate through the preprocessed data and send it to the Kafka producer line by line
for i, row in X.iterrows():
    #serialized_data = str(row)  # Convert the row to a string
    serialized_data = ','.join(map(str, row.values))
    print(f"Serialized data: {serialized_data}")
    producer.produce('detect_anomalies', serialized_data)
    time.sleep(1.5)

# Flush the producer to ensure all messages are sent
producer.flush()

----
## `anomaly_detector.py`

`anomaly_detector.py` is a Python script that performs the following tasks:

1. Import necessary libraries and modules.
2. Print a welcome message and wait for 1 second.
3. Define the on_message function, which performs the following tasks:
   1. Convert the received data to a list of strings using comma as the delimiter.
   2. Append the list of strings to the data buffer.
   3. Check if the data buffer has enough data points.
   4. If the buffer has enough data points, convert the buffer to a NumPy array of floats, predict the anomalies using the pre-trained TensorFlow model, and check if any predicted values are above the threshold.
   5. If an anomaly is detected, print an "ANOMALY" message. Otherwise, print a "BENIGN" message.
   6. Reset the data buffer.
4. Subscribe to the 'detect_anomalies' topic.
5. Consume messages from the 'detect_anomalies' topic and process them using the on_message function.

In [ ]:
# anomalay_detector.py
# https://www.phind.com/search?cache=cf139efb-38e8-4fb5-9cda-5c67194a11a6

from confluent_kafka import Consumer, Producer, KafkaError
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import time

# Print the header for the anomaly detector module
print("==================================")
print("SPECTRE - CONSUMER & ANOMALY DETECTOR MODULE")
print("==================================")
time.sleep(1)

# Load the pre-trained TensorFlow model
model = load_model('/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS_2/A/spectre_ddos_2_h5.h5')

# Print the header for the anomaly detector module
consumer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'mygroup',
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest',
    'queued.min.messages': 1  # Add this line to set the minimum number of records in the queue to 1
}

# Create a Kafka consumer instance
consumer = Consumer(consumer_conf)

# Define Kafka producer configuration for handshake with the consumer
handshake_producer_conf = {
    'bootstrap.servers': 'localhost:9092'
}

# Create a Kafka producer instance for handshake
handshake_producer = Producer(handshake_producer_conf)
handshake_producer.produce('handshake', 'READY')

# Define Kafka consumer configuration for handshake with the producer
handshake_consumer_conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'consumer_handshake_group',
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'
}

# Create a Kafka consumer instance for handshake
handshake_consumer = Consumer(handshake_consumer_conf)
handshake_consumer.subscribe(['handshake'])

# Initialize timeout counter and limit for handshake
timeout_counter = 0
timeout_limit = 10

# Perform handshake with the producer
while True:
    msg = handshake_consumer.poll(1.0)
    if msg is None:
        timeout_counter += 1
        if timeout_counter >= timeout_limit:
            print("==================================")
            print("CONNECTION FAILURE")
            print("==================================")
            exit(1)
        continue
    if msg.error():
        print(f"Handshake consumer error: {msg.error()}")
    else:
        handshake_msg = msg.value().decode('utf-8')
        if handshake_msg == 'READY':
            print("==================================")
            print("CONNECTION ESTABLISHED")
            print("==================================")
            break

# Subscribe to the 'detect_anomalies' topic
consumer.subscribe(['detect_anomalies'])

# Initialize the data buffer
received_data_buffer = []

# Consume messages and process them using the on_message function
def on_message(msg):
    global received_data_buffer
    # Choosing a higher threshold value (e.g., 0.7) will reduce the chances of benign data being misclassified as anomalies (false positives)
    # but might also result in missing some actual anomalies (false negatives). The best threshold value balances the trade-off between false positives and false negatives.
    # One approach to determine a good threshold value is to learn from past data, identifying the minimum and maximum deviations and setting the threshold accordingly, possibly with a scaling factor for flexibility.
    threshold = 0.7  # Set the threshold value for anomaly detection
    
    
    if msg.error():
        print(f"Consumer error: {msg.error()}")
    else:
        received_data_str = msg.value().decode('utf-8')  # Convert the received data to a string
        received_data_list = received_data_str.strip('[]').split(',')  # Convert the received data string to a list of strings using comma as the delimiter
        received_data_buffer.append(received_data_list)  # Append the list of strings to the buffer

        if len(received_data_buffer) == 7:
            X_received = np.array(received_data_buffer, dtype=np.float64)  # Convert the buffer to a numpy array of floats
            prediction = model.predict(X_received)
            print(f'Prediction: {prediction}')
            
            # Check if there is an anomaly and print the appropriate message
            if np.any(prediction > threshold):
                print("==================================")
                print("ANOMALY")
                print("==================================")
            else:
                print("==================================")
                print("BENIGN")
                print("==================================")
                
            received_data_buffer = []  # Reset the buffer
        else:
            # Debug: Print the received_data_str length
            #print(f"Received data length: {len(received_data_str)}")  
            
            # Debug: Print the received_data_buffer
            print(f"Received data instances: {len(received_data_buffer)}") 

# Consume messages and process them using the on_message function
while True:
    msg = consumer.poll(1.0)
    if msg is None:
        continue
    if msg.error():
        print(f"Consumer error: {msg.error()}")
    else:
        on_message(msg)

# Data Preprocess Method

In [ ]:
# Define a function to preprocess the data
def prod_datapreprocess(csv_file):
    
    # Read a CSV file and create a DataFrame
    df = pd.read_csv(csv_file)
    
    dimensions_num_for_PCA = 7
    
    # Function to clean the dataset by removing NaN, inf, and -inf values
    def clean_dataset(df):
        assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
        df.dropna(inplace=True)
        indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        return df[indices_to_keep]

    # Function to get PCA feature names
    def get_PCA_feature_names(num_of_pca_components):
        feature_names = []
        for i in range(num_of_pca_components):
            feature_names.append(f"Principal component {i+1}")
        return feature_names
    
    # Preprocess the dataset
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df_cleaned = df.copy()
    df_cleaned = clean_dataset(df_cleaned)

    df_cleaned = df_cleaned.reset_index()
    df_cleaned.drop('index', axis=1, inplace=True)

    # Saving the label attribute before dropping it
    df_labels = df_cleaned['label']
    df_cleaned.drop('label', axis=1, inplace=True)
    df_features = df_cleaned.columns.tolist()

    # Perform feature scaling
    df_scaled = StandardScaler().fit_transform(df_cleaned)
    df_scaled = pd.DataFrame(data=df_scaled, columns=df_features)

    # Performing PCA
    pca = PCA(n_components=dimensions_num_for_PCA)
    principal_components = pca.fit_transform(df_scaled)

    # Creating a DataFrame with principal components
    principal_component_headings = get_PCA_feature_names(dimensions_num_for_PCA)
    df_pc = pd.DataFrame(data=principal_components, columns=principal_component_headings)

    # Combine the principal components with the original labels
    df_final = pd.concat([df_pc, df_labels], axis=1)

    # Perform label binarization. Converts "ANOMALY" = 1 and "BENIGN" = 0.
    lb = LabelBinarizer()
    df_final['label'] = lb.fit_transform(df_final['label'])

    # Split the dataset into features (X) and labels (y)
    X = df_final.drop(['label'], axis = 1)
    y = df_final['label']

    # Returns features(X)
    return X